In [1]:
# =========================================
# INSTALL (Run once in Colab)
# =========================================
!pip install pandas numpy faker

import pandas as pd
import numpy as np
from faker import Faker
import random
import json
import string
from datetime import datetime, timedelta

fake = Faker()

# =====================================
# CONFIG (UPDATED)
# =====================================
NUM_SITES = 3
NUM_EMPLOYEES = 450
NUM_MENU = 30

NUM_MONTHS = 4
DAYS_PER_MONTH = 30
TOTAL_DAYS = NUM_MONTHS * DAYS_PER_MONTH

NUM_ORDERS = 12000
NUM_PRODUCTION = 2500
NUM_INVENTORY = 1200
NUM_WASTE = 9000
NUM_IOT = 15000

# =====================================
# ID GENERATOR
# =====================================
def gen_id(prefix):
    return prefix + "_" + ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

# =====================================
# FEEDBACK SENTENCES
# =====================================
feedback_sentences = [
    "The food was fresh and very tasty.",
    "Rice was overcooked but curry was good.",
    "Portion size was small for the price.",
    "Very delicious and well cooked meal.",
    "Too oily and spicy for my taste.",
    "Food quality was excellent today.",
    "Not fresh, tasted like yesterday food.",
    "Good taste but served cold.",
    "Perfectly cooked and well seasoned.",
    "Average taste, can be improved."
]

# =====================================
# JSON GENERATORS
# =====================================
def generate_review_json():
    return {
        "review_text": random.choice(feedback_sentences),
        "sentiment": random.choice(["positive","negative","neutral"]),
        "rating_category": random.choice(["taste","quality","portion","temperature"]),
        "review_source": random.choice(["mobile_app","kiosk","web_app"]),
        "review_time": str(fake.date_time_this_year()),
        "language": random.choice(["English","Hindi","Tamil"]),
        "would_recommend": random.choice([True, False])
    }

def generate_iot_json():
    return {
        "weight_kg": round(random.uniform(0.2, 12.5), 2),
        "battery_percent": random.randint(40, 100),
        "temperature_c": round(random.uniform(18, 45), 1),
        "humidity_percent": random.randint(30, 90),
        "device_status": random.choice(["active","idle","maintenance"]),
        "error_code": random.choice(["NONE","LOW_BATTERY","CALIBRATION_REQUIRED"]),
        "signal_strength": random.randint(60, 100),
        "firmware_version": random.choice(["v1.0","v1.1","v1.2"]),
        "zone_detected": random.choice(["Storage","Prep","Cooking","Plate_Return"]),
        "event_type": random.choice(["waste_drop","bin_removed","bin_replaced"])
    }

# =====================================
# DATE RANGE (LAST 4 MONTHS)
# =====================================
end_date = datetime.today()
start_date = end_date - timedelta(days=TOTAL_DAYS)

date_range = pd.date_range(start=start_date, end=end_date)

# =====================================
# SITE
# =====================================
site_ids = [gen_id("SITE") for _ in range(NUM_SITES)]

site = pd.DataFrame({
    "site_id": site_ids,
    "site_name": [f"Site_{i}" for i in range(NUM_SITES)],
    "location": [fake.city() for _ in range(NUM_SITES)],
    "region": [random.choice(["North","South","East","West"]) for _ in range(NUM_SITES)],
    "active_flag": [1]*NUM_SITES
})

# =====================================
# KITCHEN ZONE
# =====================================
zone_ids = [gen_id("ZN") for _ in range(NUM_SITES*4)]

kitchen_zone = pd.DataFrame({
    "zone_id": zone_ids,
    "site_id": np.repeat(site.site_id.values, 4),
    "zone_name": ["Storage","Prep","Cooking","Plate_Return"]*NUM_SITES,
    "zone_type": ["Storage","Prep","Cooking","Plate_Return"]*NUM_SITES
})

# =====================================
# EMPLOYEE (450)
# =====================================
employee_ids = [gen_id("EMP") for _ in range(NUM_EMPLOYEES)]

employee = pd.DataFrame({
    "employee_id": employee_ids,
    "employee_name": [fake.name() for _ in range(NUM_EMPLOYEES)],
    "department": [random.choice(["IT","HR","Finance","Admin","Ops"]) for _ in range(NUM_EMPLOYEES)],
    "role": [random.choice(["Staff","Manager","Lead","Supervisor"]) for _ in range(NUM_EMPLOYEES)],
    "site_id": np.random.choice(site.site_id, NUM_EMPLOYEES)
})

# =====================================
# MENU
# =====================================
menu_ids = [gen_id("MENU") for _ in range(NUM_MENU)]

menu_item = pd.DataFrame({
    "menu_item_id": menu_ids,
    "menu_name": [random.choice(["Rice","Dosa","Idli","Meals","Pizza","Burger","Pasta"]) for _ in range(NUM_MENU)],
    "category": [random.choice(["Breakfast","Lunch","Dinner","Snacks"]) for _ in range(NUM_MENU)],
    "standard_cost": np.random.randint(20,200,NUM_MENU),
    "veg_flag": np.random.choice([0,1], NUM_MENU),
    "shelf_life_days": np.random.randint(1,5,NUM_MENU)
})

# =====================================
# ATTENDANCE (4 MONTHS)
# =====================================
attendance_rows = []

for emp in employee.employee_id:
    site_val = employee.loc[employee.employee_id == emp, "site_id"].values[0]
    for d in date_range:
        attendance_rows.append([
            gen_id("ATT"),
            emp,
            site_val,
            d,
            np.random.choice([0,1], p=[0.08,0.92])
        ])

attendance = pd.DataFrame(attendance_rows, columns=[
    "attendance_id","employee_id","site_id","attendance_date","present_flag"
])

# =====================================
# ORDERS
# =====================================
orders = pd.DataFrame({
    "order_id": [gen_id("ORD") for _ in range(NUM_ORDERS)],
    "employee_id": np.random.choice(employee.employee_id, NUM_ORDERS),
    "menu_item_id": np.random.choice(menu_item.menu_item_id, NUM_ORDERS),
    "site_id": np.random.choice(site.site_id, NUM_ORDERS),
    "order_time": [fake.date_time_between(start_date=start_date, end_date=end_date) for _ in range(NUM_ORDERS)],
    "quantity": np.random.randint(1,4,NUM_ORDERS),
    "rating": np.random.randint(1,6,NUM_ORDERS),
    "review_json": [json.dumps(generate_review_json()) for _ in range(NUM_ORDERS)]
})

# =====================================
# PRODUCTION
# =====================================
production = pd.DataFrame({
    "batch_id": [gen_id("PRD") for _ in range(NUM_PRODUCTION)],
    "menu_item_id": np.random.choice(menu_item.menu_item_id, NUM_PRODUCTION),
    "site_id": np.random.choice(site.site_id, NUM_PRODUCTION),
    "production_date": [fake.date_between(start_date=start_date, end_date=end_date) for _ in range(NUM_PRODUCTION)],
    "produced_qty": np.random.randint(50,300,NUM_PRODUCTION),
    "planned_qty": np.random.randint(50,300,NUM_PRODUCTION)
})

production["overproduction_flag"] = (
    production["produced_qty"] > production["planned_qty"]
).astype(int)

# =====================================
# INVENTORY
# =====================================
inventory = pd.DataFrame({
    "inventory_lot_id": [gen_id("INV") for _ in range(NUM_INVENTORY)],
    "menu_item_id": np.random.choice(menu_item.menu_item_id, NUM_INVENTORY),
    "site_id": np.random.choice(site.site_id, NUM_INVENTORY),
    "supplier_name": [fake.company() for _ in range(NUM_INVENTORY)],
    "lot_number": [gen_id("LOT") for _ in range(NUM_INVENTORY)],
    "expiry_date": [fake.date_between(start_date=end_date, end_date=end_date+timedelta(days=120)) for _ in range(NUM_INVENTORY)],
    "received_qty": np.random.randint(100,800,NUM_INVENTORY),
    "lab_report_path": [f"/lab_reports/report_{i}.pdf" for i in range(NUM_INVENTORY)],
    "lab_report_status": np.random.choice(["PASS","FAIL"], NUM_INVENTORY),
    "lab_report_month": np.random.choice(range(1,13), NUM_INVENTORY)
})

# =====================================
# WASTE
# =====================================
waste = pd.DataFrame({
    "waste_event_id": [gen_id("WST") for _ in range(NUM_WASTE)],
    "site_id": np.random.choice(site.site_id, NUM_WASTE),
    "zone_id": np.random.choice(kitchen_zone.zone_id, NUM_WASTE),
    "menu_item_id": np.random.choice(menu_item.menu_item_id, NUM_WASTE),
    "waste_weight": np.round(np.random.uniform(0.5,12,NUM_WASTE),2),
    "waste_timestamp": [fake.date_time_between(start_date=start_date, end_date=end_date) for _ in range(NUM_WASTE)],
    "waste_reason": np.random.choice(["Overcook","Spoilage","Plate Waste"], NUM_WASTE),
    "iot_device_id": [gen_id("DEV") for _ in range(NUM_WASTE)]
})

# =====================================
# IOT RAW
# =====================================
iot_raw_data = pd.DataFrame({
    "event_id": [gen_id("EVT") for _ in range(NUM_IOT)],
    "device_id": [gen_id("DEV") for _ in range(NUM_IOT)],
    "raw_json_payload": [json.dumps(generate_iot_json()) for _ in range(NUM_IOT)],
    "event_timestamp": [fake.date_time_between(start_date=start_date, end_date=end_date) for _ in range(NUM_IOT)],
    "ingestion_time": [fake.date_time_between(start_date=start_date, end_date=end_date) for _ in range(NUM_IOT)]
})

# =====================================
# SAVE CSV
# =====================================
tables = {
    "site": site,
    "kitchen_zone": kitchen_zone,
    "employee": employee,
    "menu_item": menu_item,
    "attendance": attendance,
    "orders": orders,
    "production": production,
    "inventory": inventory,
    "waste": waste,
    "iot_raw_data": iot_raw_data
}

for name, df in tables.items():
    df.to_csv(f"{name}_4months.csv", index=False)

print("✅ 4 Months Dataset Generated Successfully!")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 20.3 MB/s eta 0:00:00
✅ 4 Months Dataset Generated Successfully!
